## Building Simple DSL's

Now that you've worked through Part 1 of this tutorial, it's time to start putting your newly acquired knowledge to use by solving more interesting problems. To that end, we'll be writing substantially more complicated macros going forward. As macros get more complicated, they often start to resemble simple [domain-specific languages](https://en.wikipedia.org/wiki/Domain-specific_language), which I'll always refer to as DSL's going forward.

Macros that operate on Julia expressions can be used to write [embedded DSL's](https://en.wikipedia.org/wiki/Domain-specific_language#External_and_Embedded_Domain_Specific_Languages) that provide alternative semantics for existing Julia syntax, even though these DSL's must still strictly conform to Julia's existing syntax rules. If you want to provide novel syntax that isn't present in Julia, you can do so by writing a macro that takes in one or more strings as inputs. Julia's [non-standard string literals](https://docs.julialang.org/en/v1/manual/metaprogramming/#Non-Standard-String-Literals-1) are examples of this approach that come with additional syntactic sugar to make them seem closely integrated into the rest of the language. This approach to custom syntax via macros that take in strings instead of AST's is closely related to [reader macros in Lisp](https://letoverlambda.com/index.cl/guest/chap4.html). We won't discuss string macros in depth here, although there's an exercise at the end that you can do to experiment with them.

In what follows, you're going to implement several DSL's. To make it easier to handle the complexity of writing DSL's from scratch, we'll generally construct a sequence of increasingly complicated DSL's that are nested inside of each other. Adding complexity incrementality is a good way to make macro authoring tractable since handling the entirety of Julia's syntax by default can be overwhelming.

Note that these DSL's are not formally specified because we won't spell out a formal grammar or specification. This is common practice for Julia DSL's, although it also accounts for many cases in which Julia DSL's have surprising edge cases that were not considered by the DSL creators. Because these DSL's are just regular macros, they conceivably could be passed arbitrarily complicated quoted code, but they often won't be capable of processing such code. Providing good error messages is one of the tricks to writing great macros, but we won't go into it in detail. The provided solutions to some of the exercises demonstrate a few approaches to producing usable error messages.

## Our First DSL: Graph Literals

Programming languages typically offer literals for many of their primitive types; Julia offers literals integers (`1`), floating point numbers (`1.0`), strings (`"foo"`), symbols (`:foo`) and more. Using these literals can make code more readable; to see why you benefit from them, imagine a language that only had string literals and required you to write `1` as something like `parse(Int, "1")`. You would probably find this pretty tedious.

Macros provide one mechanism for defining something like literals for more complex types. We're going to explore this topic by writing a macro that lets users write out graphs as macro calls. For example, we want something like:

```
@graph begin
    1 -> 2
    2 -> 3
end
```

to expand to the following substantially more verbose graph constructor:


```
let edges = ((1, 2), (2, 3)), g = LightGraphs.SimpleDiGraph(3)
    for e in edges
        LightGraphs.add_edge!(g, e[1], e[2])
    end
    g
end
```

We'll also want to support undirected graphs like the following:

```
@graph begin
    1 - 2
    2 - 3
end
```

We won't support mixed graphs that contain both directed and undirected edges.

### Implementation Stragegy

To get started, we'll restrict our focus to directed graphs and then cycle back to support undirected graphs. The high-level approach will be:

1. Assume we're being passed a block in which each expression is a directed edge. Anything else will be an error on the user's part, so we'll throw an error if it occurs.
2. Given each of directed edge expressions, we'll transform it into a form that only contains information needed for the semantics of defining a graph. In our case, `:(1 -> 2)` will be transformed into the tuple `(1, 2)` assuming background information about the edge being directed.
3. Given all of the edges represented as tuples, we'll generate the suggested graph constructor expression based on the specified edges.

### Step 1: Processing a Block

We'll do the bare minimum needed to get something working and then suggest improvements in the exercises. To start, we'll write code to process a full graph as a block expression:

1. We'll assert the passed in block really is a block.
2. We'll output the edges and nodes as `Vector{NTuple{2, Int}}` and `Set{Int}` respectively.
3. We'll just ignore line numbers.
4. We'll assume the block only contains valid directed edge expressions.
5. We'll extract the edge from the expression.
6. We'll return the edges and nodes.

In [1]:
function extract_graph(e::Expr)
    @assert e.head == :block
    edges = NTuple{2, Int}[]
    nodes = Set{Int}()
    for ex in e.args
        if isa(ex, LineNumberNode)
            continue
        end
        edge = extract_edge(ex)
        push!(edges, edge)
        push!(nodes, edge[1])
        push!(nodes, edge[2])
    end
    edges, nodes
end

extract_graph (generic function with 1 method)

### Step 2: Processing an Edge

To make `extract_graph` work, we need to implement `extract_edge`. This is fairly simple code that mostly requires dealing with the specific format of `Expr` objects that represent anonymous functions. Because we need to make many assumptions, we'll include a lot of `@assert` calls that should be turned into proper error messages that users could use to fix their code.

In [2]:
function extract_edge(e::Expr)
    @assert e.head == :->
    @assert isa(e.args[1], Integer)
    e′ = e.args[2]
    @assert e′.head == :block
    @assert length(e′.args) == 2
    @assert isa(e′.args[2], Integer)
    (e.args[1], e′.args[2])
end

extract_edge (generic function with 1 method)

Let's test our code on a few cases to see that it can handle the simplest examples we can think of:

In [3]:
extract_edge(:(1 -> 2))

(1, 2)

In [4]:
extract_edge(:(1 -> 3))

(1, 3)

In [5]:
extract_edge(:(2 -> 1))

(2, 1)

In [6]:
extract_graph(:(
    begin
        1 -> 2
        2 -> 3
    end
))

([(1, 2), (2, 3)], Set([2, 3, 1]))

### Step 3: Write the Macro

Now that we have the core logic in place for transforming a block expression representing a graph into a condensed form, writing the `@graph` macro is very simple. To make the macro easier to read, we produce our final expression using quasiquotation splicing:

In [7]:
macro graph(e)
    edges, nodes = extract_graph(e)
    n = length(nodes)
    quote
        import LightGraphs
        let edges = $edges, g = LightGraphs.SimpleDiGraph($n)
            for e in edges
                LightGraphs.add_edge!(g, e[1], e[2])
            end
            g
        end
    end
end

@graph (macro with 1 method)

Now we can confirm that our macro works:

In [8]:
@graph begin
    1 -> 2
    2 -> 3
end

{3, 2} directed simple Int64 graph

### Exercises

1. Extend the DSL to support multiple edges per line like the following:

```
1 -> 2 -> 3
```

2. Support writing edges in the other direction:

```
1 <- 2
```

3. Mix multiple edges per line with support for both left and right directed edges:

```
1 <- 2 -> 3
```

4. Suppport undirected graphs using `1 - 2` to represent an edge instead of `1 -> 2`.

5. Think through the issues for and against allowing the macro to decide for itself whether the output graph is directed or undirected. Instead of allowing the macro to decide, do you want to use two distinct macros called `@graph` and `@digraph`? Does type stability matter for macros that act like literals? Why or why not?

6. Ensure that your macro emits useful error information. At the least, provide error messages that describe which expression caused an error, what kind of error was encountered and what was expected to occur in that expression. Even better, add line numbers to improve reporting.

7. Much more ambitious exercise: make `@graph` into a string macro that processes raw strings that you parse with a custom parser. Invent a piece of graph syntax that Julia's normal syntax doesn't support.

8. Convince yourself `@graph` can't be a function unless you pass in strings. What are some of the reasons that something like the following wouldn't work?

```
graph(1 -> 2, 2 -> 3)
```

8. (Cont.) What problems would we hit if we tried this instead?

```
graph(1 => 2, 2 => 3)
```

## Named Tuple Rand

Now that we've built our first simple DSL, let's explore a richer space. In what follows, we'll build a DSL that lets users describe a probabilistic model using the notation found in probabistic programming languages (aka PPL's) like BUGS, JAGS, Stan or Turing. But, unlike a true PPL, we're only going to use this model specification to generate a single sample from the model, which we'll return as a named tuple.

Just this simplified topic is very deep and we'll explore only a piece of it in what follows; there's essentially no limit to how rich the modeling language could be made if you decided to invest more time into it.

### Exercise 1: Named Tuple RNG without Dependencies

To get started write a macro `@rng` that takes in a simple model with no dependencies between variables and generates a named tuple. For example, the following model:

```
@rng begin
    mu ~ Normal(0, 1)
    sigma ~ Gamma(1, 1)
end
```

should generate the following named tuple:

```
(
    mu = rand(Distributions.Normal(0, 1)),
    sigma = rand(Distributions.Gamma(1, 1)),
)
```

Do this for yourself. The logic here is quite similar to the graph literals in the previous section. If you get stuck, consult the [solutions](https://github.com/johnmyleswhite/julia_tutorials/tree/master/solutions/).

### Exercise 2: Named Tuple RNG with Dependencies

Now that you have something simple working, let's introduce dependencies between variables. To make things simpler for now, you can assume that the model block is written in the order in which calls to `rand` must happen:

```
mu ~ Normal(0, 1)
sigma ~ Gamma(1, 1)
x ~ Normal(mu, sigma)
```

Note that you'll need to deal with scoping issues here because named tuples can't refer to previous tuple elements (unlike keyword arguments).

### Exercise 3: Named Tuple RNG with Out-of-Order Dependencies

Once you've finished Exercise 2, modify your code to support a block in which variables are defined in an arbitrary order, requiring you, the macro author, to topologically sort them before emitting calls to `rand`.

```
x ~ Normal(mu, sigma)
mu ~ Normal(0, 1)
sigma ~ Gamma(1, 1)
```

### Exercise 4: Support Expressions as Distribution Parameters

None of the examples so far passed in expressions as arguments to the distribution constructors. Add support for things like:

```
x ~ Normal(mu + 1, sigma)
mu ~ Normal(0, 1)
sigma ~ Gamma(1, 1)
```

### Exercise 5: Introducing Branches

Enrich your modeling language by support `if`/`else` expressions:

```
x ~ Bernoulli(0.5)
if x == 1
    y ~ Normal(+1, 1.0)
else
    y ~ Normal(-1, 1.0)
end
```

Sometimes this can be done trivially using existing dependencies if the parameters of a distribution are arbitrary expressions (as in the following example), but this is not sufficient in the distribution itself varies across the branches of the `if`/`else` expression:

```
x ~ Bernoulli(0.5)
y ~ Normal(+1 * x + -1 * (1 - x), 1.0)
```

### Exercise 6: Support Statically Bounded For Loops

Further enrich your language by adding support for static for loops whose loop bounds are known at macro expansion time like the following example in which the bounds are statically known to be `1` and `10`:

```
mu ~ Normal(0, 1)
sigma ~ Gamma(1, 1)
for i in 1:10
    x[i] ~ Normal(mu, sigma)
end
```

You should decide whether you want to generate elements of the tuple like `Symbol("x[1]")` or whether `x` should be a vector. What are arguments for and against each approach?

There's also an interesting question here about the kind of code you want to emit: do you want to emit code uses for loops? Or do you want to emit code in which each variable defined over the course of the loop generates its own expression? The latter is called [unrolling a loop](https://en.wikipedia.org/wiki/Loop_unrolling).

### Exercise 7: Support Dynamically Bounded For Loops

Go a step further and support dynamic for loops whose bounds are **not** known at macro expansion time like the following in which the uppper loop bound is a variable:

```
mu ~ Normal(0, 1)
sigma ~ Gamma(1, 1)
for i in 1:n
    x[i] ~ Normal(mu, sigma)
end
```

Because this computation depends on information that cannot be known at compile time, you need to produce a function of `n` that evaluates to a named tuple; this is introduces a breaking change relative to your previous work.

Note that this kind of dynamic bounds completely rules out manual unrolling at macro compile time.

### Exercise 8: Non-IID Loops

So far all of the loops we've considered (whether statically bound or dynamically bound) have not introduced dependencies between the output values generated by the loop. Extend your language (if necessary) to support dependencies like the following one:

```
mu ~ Normal(0, 1)
sigma ~ Gamma(1, 1)
x[1] ~ Normal(mu, sigma)
for i in 2:n
    x[i] ~ x[i - 1] + Normal(mu, sigma)
end
```

## More Ideas for Macros and DSL's to Implement

If you're worked through these DSL's and want to work on more projects, here's a few to consider. Some of these are really just more complicated macros rather than DSL's.

1. **Symbolic Differentiation**

Write a macro that uses [Calculus.jl](https://github.com/JuliaMath/Calculus.jl) to perform symbolic differentiation on expressions to generate their derivatives. Your macro should look something like:

```
@derivative(x + sin(exp(x)))
```

and it should generate output like:

```
x -> 1 + cos(exp(x)) * exp(x)
```

2. **Embedded Lisp**

As noted earlier, you can support novel syntax using string macros. Based on ideas [from Peter Norvig's blog](https://norvig.com/lispy.html), implement a basic version of Lispy using a Julia string macro:

```
lisp"(+ 1 (* 2 3))"
```

3. **Curve Plotting**

Write a macro that plots a curve specified in terms of the assumed variable `x`. Make sure that you both make use of the expression as a function while also representing the expression incorrectly in the axis labels for your plot. For example, be sure the labels for these two calls produce different outputs even though they define the same functions:

```
@curve(sin(x + y), xlim = (-1, 1), ylim = (-1, 1))
```

```
@curve(sin(y + x), xlim = (-1, 1), ylim = (-1, 1))
```

4. **Model Transformer DSL**

One of the main uses of [non-standard evaluation](http://adv-r.had.co.nz/Computing-on-the-language.html) in the R programming language is to support [Wilkinson notation](https://www.jstor.org/stable/2346786?seq=1) for transforming tabular data into design matrices for modeling. Implement something like:

```
@transformer(z ~ 1 + x + factor(y))
```

This should generate a transformer object that, when applied to a DataFrame, would generate the appropriate matrix. If you're unfamiliar with linear models, you should probably skip this exercise as the modeling issues are much deeper than the macro authoring issues.